<a href="https://colab.research.google.com/github/Damjanv1/Sideprojects/blob/main/Euroleague_chant_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# First remove existing PRAW installation
!pip uninstall -y praw prawcore
# Install latest version
!pip install --upgrade praw

import praw
from getpass import getpass
import sys
print(f"Python version: {sys.version}")
print(f"PRAW version: {praw.__version__}")

try:
    reddit = praw.Reddit(
        client_id="FExpDe13al_W6thva_RTqA",
        client_secret="cmwqMmrQom1V9tqeoVyOejRIijeijQ",
        user_agent="Script by u/Euroleaguechantsbot",
        username="Euroleaguechantsbot",  # Directly using username for testing
        password=getpass("Reddit Password: ")  # Still keeping password secure
    )

    # Test authentication
    print("Attempting to authenticate...")
    me = reddit.user.me()
    print(f"Successfully logged in as: {me}")

    # Try to access subreddit as a test
    print("Attempting to access r/Euroleague...")
    subreddit = reddit.subreddit("Euroleague")
    print(f"Successfully accessed subreddit: {subreddit.display_name}")

except Exception as e:
    print("\nError occurred:")
    print(f"Error type: {type(e)}")
    print(f"Error details: {str(e)}")

Found existing installation: praw 7.8.1
Uninstalling praw-7.8.1:
  Successfully uninstalled praw-7.8.1
Found existing installation: prawcore 2.4.0
Uninstalling prawcore-2.4.0:
  Successfully uninstalled prawcore-2.4.0
  Using cached praw-7.8.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached prawcore-2.4.0-py3-none-any.whl.metadata (5.0 kB)
Using cached praw-7.8.1-py3-none-any.whl (189 kB)
Using cached prawcore-2.4.0-py3-none-any.whl (17 kB)
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
PRAW version: 7.8.1
Reddit Password: ··········


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Attempting to authenticate...
Successfully logged in as: Euroleaguechantsbot
Attempting to access r/Euroleague...
Successfully accessed subreddit: Euroleague


In [11]:
import praw
import random
import time
from datetime import datetime, timedelta

# Partizan chants remain the same as before
PARTIZAN_CHANTS = [
    (
        """Da volim crno-bele
ponosno kažem svima
volim to slavno ime i
divim se samo njima
Na svetu ništa veće
ne može da postoji
nego što je naša ljubav
prema crno-beloj boji
Ljubav prema klubu
ne može da prestane
dok živim klicaću njemu
volim te Partizane!!""",
        """That I love black-n-whites
I proudly say it to everyone
I love that famous name
and I only admire them.
In the world,
nothing bigger can exist,
Than our love
for black and white colour.
Love for the club
Can not ever stop,
While I am alive
I will shout to him:
"I Love you, Partizan\""""
    ),
    (
        """Ti si ponos Beograda,
Ti si ponos zemlje moje
Ti si ponos Srbije,
Crno-beliii!""",
        """You are the pride of Belgrade,
You are the pride of my country!
You are the pride of Serbia,
Black-n-Whites!"""
    ),
    (
        """Partizane jedini,
Mi za tebe zivimo,
Samo tebe volimo,
I tebi se divimo,
a kad dodju nevolje,
I kad dodje tuga,
Uvek ce te voleti,
GROBARI SA JUGA!!!
Volim Partizan!""",
        """Partizan one-and-only
We live for you,
We only love you
And we admire you.
When the troubles come
and when the sorrow comes
Always will love you,
Grobari from the SOUTH!
I love Partizan!"""
    )
]

def create_reddit_instance():
    return praw.Reddit(
        client_id="FExpDe13al_W6thva_RTqA",
        client_secret="cmwqMmrQom1V9tqeoVyOejRIijeijQ",
        user_agent="Script by u/Euroleaguechantsbot",
        username="Euroleaguechantsbot",
        password=input("Enter bot password: ")
    )

def did_partizan_win(title, text):
    """Check if Partizan won by parsing the score from the markdown table."""
    if "Partizan" not in title:
        print("Not a Partizan game")
        return False

    lines = text.split('\n')
    partizan_score = None
    opponent_score = None

    try:
        # Find the table that contains scores
        for i, line in enumerate(lines):
            if "|TEAM|SCORE|" in line:
                print("Found score table")
                # Skip the separator line (|:-|:-:|)
                score_lines = lines[i+2:i+4]  # Get the next two lines after the header

                for line in score_lines:
                    if not line.strip() or '|-' in line:  # Skip empty lines or separator lines
                        continue

                    parts = line.split('|')
                    if len(parts) >= 3:  # Should be ['', 'TEAM', 'SCORE', '']
                        if "Partizan" in parts[1]:
                            partizan_score = int(parts[2].strip())
                            print(f"Found Partizan score: {partizan_score}")
                        else:
                            try:
                                opponent_score = int(parts[2].strip())
                                print(f"Found opponent score: {opponent_score}")
                            except ValueError:
                                continue

        if partizan_score is not None and opponent_score is not None:
            print(f"Final scores - Partizan: {partizan_score}, Opponent: {opponent_score}")
            return partizan_score > opponent_score

    except Exception as e:
        print(f"Error parsing scores: {e}")

    print("Could not find scores")
    return False

def format_chant(chant):
    serbian, english = chant
    return f"""# Grobari Victory Chant 🖤⚪

## Original (Serbian):
{serbian}

## English Translation:
{english}

---
^(I am a bot that celebrates your favourite Euroleague team victories with their chants! | 🤖)"""

def check_historical_posts(reddit, days_back=2):
    """Check posts from the last 2 days for Partizan victories."""
    subreddit = reddit.subreddit("Euroleague")
    cutoff_time = int((datetime.utcnow() - timedelta(days=days_back)).timestamp())

    print(f"\nChecking posts from the last {days_back} days...")

    for submission in subreddit.new(limit=50):
        try:
            if submission.created_utc < cutoff_time:
                break

            if "POST MATCH THREAD" in submission.title.upper():
                print(f"\nFound historical match thread: {submission.title}")
                print(f"Post content:\n{submission.selftext[:500]}...")

                already_commented = any(comment.author == reddit.user.me()
                                     for comment in submission.comments)

                if not already_commented and did_partizan_win(submission.title, submission.selftext):
                    print("Found historical Partizan victory! Posting chant...")
                    chant = random.choice(PARTIZAN_CHANTS)
                    comment = format_chant(chant)
                    submission.reply(comment)
                    print("Successfully posted chant!")
                    time.sleep(5)
        except Exception as e:
            print(f"Error processing post: {e}")
            continue

def run_bot():
    print("Starting bot...")
    reddit = create_reddit_instance()
    subreddit = reddit.subreddit("Euroleague")

    print(f"Successfully logged in as: {reddit.user.me()}")

    # First check historical posts
    check_historical_posts(reddit, days_back=2)

    print("\nNow monitoring for new posts...")
    already_processed = set()

    while True:
        try:
            for submission in subreddit.new(limit=10):
                if submission.id in already_processed:
                    continue

                if "POST MATCH THREAD" in submission.title.upper():
                    print(f"\nFound new match thread: {submission.title}")

                    already_commented = any(comment.author == reddit.user.me()
                                         for comment in submission.comments)

                    if not already_commented and did_partizan_win(submission.title, submission.selftext):
                        print("Partizan won! Posting chant...")
                        chant = random.choice(PARTIZAN_CHANTS)
                        comment = format_chant(chant)
                        submission.reply(comment)
                        print("Successfully posted chant!")

                    already_processed.add(submission.id)

            print(".", end="", flush=True)
            time.sleep(60)

        except Exception as e:
            print(f"\nError occurred: {e}")
            print("Waiting 5 minutes before retrying...")
            time.sleep(300)

if __name__ == "__main__":
    run_bot()

Starting bot...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Successfully logged in as: Euroleaguechantsbot

Checking posts from the last 2 days...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




Found historical match thread: POST MATCH THREAD: Virtus Bologna - Alba Berlin [EuroLeague Regular Season, Round 13]
Post content:
**Event Date:** December 04, 2024 20:45 CET


**Stadium:** VIRTUS SEGAFREDO ARENA


**Attendance:** 6104


**Referees:** HORDOV, TOMISLAV ; VILIUS, GYTIS; BALAK, AMIT

----

|TEAM|SCORE|
|:-|:-:|
|[Virtus Bologna](https://www.euroleaguebasketball.net/euroleague/teams/virtus-segafredo-bologna/vir/?season=2023-24)|88|
|[Alba Berlin](https://www.euroleaguebasketball.net/euroleague/teams/alba-berlin/ber/?season=2023-24)|90|

----

|TEAM|1|2|3|4|OT 1|
|:-|:-:|:-:|:-:|:-:|:-:|
|[Virtus Bologna](https:...


...

.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



KeyboardInterrupt: 

In [13]:
import praw
import random
import time
from datetime import datetime, timedelta

# Keep your exact TEAM_CHANTS dictionary here [unchanged]
TEAM_CHANTS = {
    "Partizan": {
        "colors": "🖤⚪",
        "fan_group": "Grobari",
        "chants": [
            (
                """Da volim crno-bele
ponosno kažem svima
volim to slavno ime i
divim se samo njima
Na svetu ništa veće
ne može da postoji
nego što je naša ljubav
prema crno-beloj boji
Ljubav prema klubu
ne može da prestane
dok živim klicaću njemu
volim te Partizane!!""",
                """That I love black-n-whites
I proudly say it to everyone
I love that famous name
and I only admire them.
In the world,
nothing bigger can exist,
Than our love
for black and white colour.
Love for the club
Can not ever stop,
While I am alive
I will shout to him:
"I Love you, Partizan\""""
            ),
            (
                """Ti si ponos Beograda,
Ti si ponos zemlje moje
Ti si ponos Srbije,
Crno-beliii!""",
                """You are the pride of Belgrade,
You are the pride of my country!
You are the pride of Serbia,
Black-n-Whites!"""
            ),
            (
                """Partizane jedini,
Mi za tebe zivimo,
Samo tebe volimo,
I tebi se divimo,
a kad dodju nevolje,
I kad dodje tuga,
Uvek ce te voleti,
GROBARI SA JUGA!!!
Volim Partizan!""",
                """Partizan one-and-only
We live for you,
We only love you
And we admire you.
When the troubles come
and when the sorrow comes
Always will love you,
Grobari from the SOUTH!
I love Partizan!"""
            )
        ]
    },
    "Crvena Zvezda": {
        "colors": "🔴⚪",
        "fan_group": "Delije",
        "chants": [
            (
                """Za večnu slavu,
za pobedu,
za samo našu Zvezdu Crvenu,
pesma nek grmi odjekuje,
nepred Zvezdo,
ludo volim te!""",
                """For eternal glory,
For victory,
For our Red Star alone,
Let the song thunder and echo,
Forward Red Star,
I love you madly!"""
            )
        ]
    },
    "Panathinaikos": {
        "colors": "💚⚪",
        "fan_group": "Gate 13",
        "chants": [
            (
                """Voodoo-Voodoo,
Voodoo-Voodoo kai prasini magia (oooooooo),
to kipelo to ferame apo ti germania (pao ole)""",
                """Voodoo-Voodoo,
Voodoo-Voodoo and green magic,
The cup we brought from Germany"""
            ),
            (
                """Chorto magiko,
dose mou ligo gia na pio,
ton pao mou na onirefto,
kai na fonakso ston theo,
panatha mou se agapo,
san iroini san skliro narkotiko,
san to chasis to lsd,
gia sena pao mastouroni oli i gi.

Panatha mou se agapo,
opou kai an paizeis tha se akoloutho,
pao edo,
pao ekei,
opou kai an paizeis thamaste mazi""",
                """Magic leaf,
Give me some to consume,
To dream of my Pao,
And shout to God,
'My Panatha I love you,
Like heroine like a strong drug,
Like weed LSD,
For you Panatha the whole globe is high.

My Panatha I love you,
Wherever you play I will follow,
Pao here,
Pao there,
Wherever you play we would be together"""
            ),
            (
                """Sto Belegradi tha einai oraia,
me to ogdoo parea""",
                """In Belgrade,
It will be beautiful together with the eighth."""
            ),
            (
                """Παναθηναϊκέ μου τρελέ,
τρέλα μου πράσινη,
για σένα ζω και αναπνέω,
μόνο για σένα τραγουδώ!""",
                """My crazy Panathinaikos,
My green madness,
For you I live and breathe,
Only for you I sing!"""
            )
        ]
    },
    "Olympiacos": {
        "colors": "🔴⚪",
        "fan_group": "Gate 7",
        "chants": [
            (
                """Θρύλε σ' αγαπώ παντού θα σ' ακολουθώ,
στα εύκολα και στα δύσκολα μαζί σου θα βρεθώ.
Είσαι η ζωή μου όλη, Ολυμπιακέ,
και θα σε υπερασπίζομαι ως το τέλος της ζωής!""",
                """Legend I love you,
Everywhere I'll follow you,
In easy times and hard times I'll be with you.
You are my whole life, Olympiacos,
And I will defend you until the end of life!"""
            )
        ]
    }
}

def create_reddit_instance():
    return praw.Reddit(
        client_id="FExpDe13al_W6thva_RTqA",
        client_secret="cmwqMmrQom1V9tqeoVyOejRIijeijQ",
        user_agent="Script by u/Euroleaguechantsbot",
        username="Euroleaguechantsbot",
        password=input("Enter bot password: ")
    )

def format_chant(team_name, chant):
    """Format the chant for posting with proper line breaks."""
    original, translation = chant
    team_info = TEAM_CHANTS[team_name]

    return f"""# {team_info['fan_group']} Victory Chant {team_info['colors']}

## Original:
{original}

## English Translation:
{translation}

---
^(I am a bot that celebrates Euroleague victories with traditional chants! #EveryGameMatters!|)"""

def get_team_from_title(title):
    """Extract team name from post title with variation handling."""
    # Direct mapping of variations to correct team names
    team_variations = {
        "Panathinaikos": ["Panathinaikos", "PAO", "Pao", "Panathinaikos BC"],
        "Partizan": ["Partizan", "Partizan Mozzart Bet", "Partizan Belgrade"],
        "Crvena Zvezda": ["Crvena Zvezda", "Red Star", "Zvezda"],
        "Olympiacos": ["Olympiacos", "Olympiakos", "Olympiacos BC"]
    }

    # First check for variations
    for team, variations in team_variations.items():
        if any(variant in title for variant in variations):
            print(f"Matched team {team} in title: {title}")
            return team

    # Fallback to direct check
    for team in TEAM_CHANTS.keys():
        if team in title:
            return team
    return None

def did_team_win(title, text, team_name):
    """Check if a specific team won by parsing the score from the markdown table."""
    # Get variations for the team name
    team_variations = {
        "Panathinaikos": ["Panathinaikos", "PAO", "Pao", "Panathinaikos BC"],
        "Partizan": ["Partizan", "Partizan Mozzart Bet", "Partizan Belgrade"],
        "Crvena Zvezda": ["Crvena Zvezda", "Red Star", "Zvezda"],
        "Olympiacos": ["Olympiacos", "Olympiakos", "Olympiacos BC"]
    }

    # Check if any variation of the team name is in the title
    if not any(variant in title for variant in team_variations.get(team_name, [team_name])):
        return False

    lines = text.split('\n')
    team_score = None
    opponent_score = None

    try:
        for i, line in enumerate(lines):
            if "|TEAM|SCORE|" in line:
                print(f"Found score table for {team_name}")
                score_lines = lines[i+2:i+4]

                for line in score_lines:
                    if not line.strip() or '|-' in line:
                        continue

                    parts = line.split('|')
                    if len(parts) >= 3:
                        # Check if any variation of the team name is in the line
                        if any(variant in parts[1] for variant in team_variations.get(team_name, [team_name])):
                            team_score = int(parts[2].strip())
                            print(f"Found {team_name} score: {team_score}")
                        else:
                            try:
                                opponent_score = int(parts[2].strip())
                                print(f"Found opponent score: {opponent_score}")
                            except ValueError:
                                continue

        if team_score is not None and opponent_score is not None:
            print(f"Final scores - {team_name}: {team_score}, Opponent: {opponent_score}")
            return team_score > opponent_score

    except Exception as e:
        print(f"Error parsing scores: {e}")

    return False

def check_historical_posts(reddit, days_back=2):
    """Check posts from the last 2 days for team victories."""
    subreddit = reddit.subreddit("Euroleague")
    cutoff_time = int((datetime.utcnow() - timedelta(days=days_back)).timestamp())

    print(f"\nChecking posts from the last {days_back} days...")

    for submission in subreddit.new(limit=50):
        try:
            if submission.created_utc < cutoff_time:
                break

            if "POST MATCH THREAD" in submission.title.upper():
                print(f"\nFound historical match thread: {submission.title}")
                team_name = get_team_from_title(submission.title)

                if team_name:
                    print(f"Found team: {team_name}")

                    already_commented = any(comment.author == reddit.user.me()
                                         for comment in submission.comments)

                    if not already_commented and did_team_win(submission.title, submission.selftext, team_name):
                        print(f"Found {team_name} victory! Posting chant...")
                        chant = random.choice(TEAM_CHANTS[team_name]["chants"])
                        comment = format_chant(team_name, chant)
                        submission.reply(comment)
                        print("Successfully posted chant!")
                        time.sleep(5)
        except Exception as e:
            print(f"Error processing post: {e}")
            continue

def run_bot():
    print("Starting Euroleague Chants Bot...")
    reddit = create_reddit_instance()
    subreddit = reddit.subreddit("Euroleague")

    print(f"Successfully logged in as: {reddit.user.me()}")

    # First check historical posts
    check_historical_posts(reddit, days_back=2)

    print("\nNow monitoring for new posts...")
    already_processed = set()

    while True:
        try:
            for submission in subreddit.new(limit=10):
                if submission.id in already_processed:
                    continue

                if "POST MATCH THREAD" in submission.title.upper():
                    team_name = get_team_from_title(submission.title)

                    if team_name:
                        print(f"\nFound {team_name} match thread: {submission.title}")

                        already_commented = any(comment.author == reddit.user.me()
                                             for comment in submission.comments)

                        if not already_commented and did_team_win(submission.title, submission.selftext, team_name):
                            print(f"{team_name} won! Posting chant...")
                            chant = random.choice(TEAM_CHANTS[team_name]["chants"])
                            comment = format_chant(team_name, chant)
                            submission.reply(comment)
                            print("Successfully posted chant!")

                    already_processed.add(submission.id)

            print(".", end="", flush=True)
            time.sleep(60)

        except Exception as e:
            print(f"\nError occurred: {e}")
            print("Waiting 5 minutes before retrying...")
            time.sleep(300)

if __name__ == "__main__":
    run_bot()

Starting Euroleague Chants Bot...
Enter bot password: Zzidane#10


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Successfully logged in as: Euroleaguechantsbot

Checking posts from the last 2 days...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




Found historical match thread: POST MATCH THREAD: Virtus Bologna - Alba Berlin [EuroLeague Regular Season, Round 13]

Found historical match thread: POST MATCH THREAD: Valencia - Hapoel Jerusalem [EuroCup Regular Season, Round 10]

Found historical match thread: POST MATCH THREAD: Olimpia Milano - Crvena Zvezda [EuroLeague Regular Season, Round 13]
Matched team Crvena Zvezda in title: POST MATCH THREAD: Olimpia Milano - Crvena Zvezda [EuroLeague Regular Season, Round 13]
Found team: Crvena Zvezda


...

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



.

KeyboardInterrupt: 